In [3]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

In [4]:
df = pd.read_csv('total_data(2016제외) (1).csv')

### xdata로 xgb로 결정한 feature를 넣어서 Elastic
- elasticnet으로 xgb로 결정한 feature를 넣어 모델생성

In [5]:
# 필요 패키지 로드
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error

In [6]:
# y값인 q1-q5가 결측인 2020년 데이터 제거
a = df[0:-82]
a

jur_stn       year  report_sx_남성  report_sx_불상  report_sx_여성  \
0     서울중부경찰서  2017(상반기)       13404.0        3239.0        5931.0   
1     서울종로경찰서  2017(상반기)       11579.0        2560.0        5534.0   
2    서울남대문경찰서  2017(상반기)        9093.0        1855.0        3551.0   
3    서울서대문경찰서  2017(상반기)       18441.0       10029.0       10298.0   
4     서울혜화경찰서  2017(상반기)       10829.0        2989.0        5038.0   
..        ...        ...           ...           ...           ...   
241   창원서부경찰서  2019(하반기)       12630.0        2990.0        7151.0   
242   마산중부경찰서  2019(하반기)       12223.0        2552.0        6489.0   
243   마산동부경찰서  2019(하반기)       15177.0        3190.0        7647.0   
244     진주경찰서  2019(하반기)       21840.0        5262.0       11455.0   
245     진해경찰서  2019(하반기)       11600.0        2442.0        6278.0   

     report_sx_total  vio_cnt  mur_rob_cnt  car_cnt  5m_crm_yn  ...  \
0            22574.0   1324.0          0.0   1412.0     3275.0  ...   
1            19674.0   1102.0          0.0    867.0     2914.0  ...   
2            14499.0    949.0          0.0    736.0     2506.0  ...   
3            38769.0   1924.0          0.0   1788.0     4267.0  ...   
4            18857.0   1127.0          0.0    899.0     2864.0  ...   
..               ...      ...          ...      ...        ...  ...   
241          22771.0   1460.0          2.0   1964.0     2142.0  ...   
242          21264.0   1265.0          5.0   1584.0     1369.0  ...   
243          26014.0   1715.0          8.0   1910.0     1920.0  ...   
244          38557.0   2552.0          0.0   3570.0     2876.0  ...   
245          20320.0   1444.0          0.0   1692.0     1354.0  ...   

     single_f_2030  single_f_4050  single_f_o60  adult  bell    q1    q2  \
0           2128.0         1230.5        1635.5    0.0   0.0  75.2  76.5   
1           2703.0         1378.5        1782.5    2.0   2.0  81.6  82.4   
2           2128.0         1230.5        1635.5    0.0   0.0  77.3  79.3   
3          12420.0         4496.0        7066.0    0.0   0.0  78.0  80.0   
4           2703.0         1378.5        1782.5    0.0   0.0  78.6  80.6   
..             ...            ...           ...    ...   ...   ...   ...   
241         2900.0         4492.0        5574.0    1.0   1.0  79.1  79.5   
242         2216.0         2837.0        7186.0    3.0   3.0  80.8  80.0   
243         1496.0         2939.0        6166.0    1.0   1.0  75.0  76.5   
244         6026.0         5205.0       11823.0    8.0   8.0  78.7  79.6   
245         1782.0         2592.0        4931.0    3.0   3.0  76.6  79.2   

       q3    q4    q5  
0    69.9  65.9  72.6  
1    69.1  62.7  72.2  
2    68.8  61.0  71.7  
3    67.0  66.3  71.4  
4    70.7  65.2  75.4  
..    ...   ...   ...  
241  68.5  71.2  76.0  
242  70.8  73.2  76.5  
243  69.4  68.9  73.1  
244  70.3  74.6  75.6  
245  67.0  70.9  73.5  

[246 rows x 75 columns]

In [7]:
# 경찰서와 연도 데이터 제거
a.drop(columns = ['jur_stn', 'year'], inplace = True)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [8]:
a_ = a.drop(columns = ['q1', 'q2', 'q3', 'q4', 'q5'])

In [12]:
a_

report_sx_남성  report_sx_불상  report_sx_여성  report_sx_total  vio_cnt  \
0         13404.0        3239.0        5931.0          22574.0   1324.0   
1         11579.0        2560.0        5534.0          19674.0   1102.0   
2          9093.0        1855.0        3551.0          14499.0    949.0   
3         18441.0       10029.0       10298.0          38769.0   1924.0   
4         10829.0        2989.0        5038.0          18857.0   1127.0   
..            ...           ...           ...              ...      ...   
241       12630.0        2990.0        7151.0          22771.0   1460.0   
242       12223.0        2552.0        6489.0          21264.0   1265.0   
243       15177.0        3190.0        7647.0          26014.0   1715.0   
244       21840.0        5262.0       11455.0          38557.0   2552.0   
245       11600.0        2442.0        6278.0          20320.0   1444.0   

     mur_rob_cnt  car_cnt  5m_crm_yn  theft_violence_x  robber_murder_x  ...  \
0            0.0   1412.0     3275.0             572.0              4.0  ...   
1            0.0    867.0     2914.0             525.0              2.0  ...   
2            0.0    736.0     2506.0             413.0              0.0  ...   
3            0.0   1788.0     4267.0             674.0              0.0  ...   
4            0.0    899.0     2864.0             427.0              4.0  ...   
..           ...      ...        ...               ...              ...  ...   
241          2.0   1964.0     2142.0             363.0              2.0  ...   
242          5.0   1584.0     1369.0             151.0              3.0  ...   
243          8.0   1910.0     1920.0             312.0              6.0  ...   
244          0.0   3570.0     2876.0             537.0              0.0  ...   
245          0.0   1692.0     1354.0             187.0              0.0  ...   

     single_2030  single_4050  single_o60  single_f_합계  single_f_u20  \
0         4310.5       2954.5      2567.5       5169.0         175.0   
1         5483.5       3220.5      2861.5       6022.0         158.0   
2         4310.5       2954.5      2567.5       5169.0         175.0   
3        23040.0       9687.0      9894.0      24527.0         545.0   
4         5483.5       3220.5      2861.5       6022.0         158.0   
..           ...          ...         ...          ...           ...   
241       8529.0      10946.0      8555.0      13016.0          50.0   
242       5895.0       6580.0     10395.0      12431.0         192.0   
243       3791.0       7160.0      9299.0      10643.0          42.0   
244      15178.0      11882.0     16273.0      23362.0         308.0   
245       6316.0       7359.0      7241.0       9331.0          26.0   

     single_f_2030  single_f_4050  single_f_o60  adult  bell  
0           2128.0         1230.5        1635.5    0.0   0.0  
1           2703.0         1378.5        1782.5    2.0   2.0  
2           2128.0         1230.5        1635.5    0.0   0.0  
3          12420.0         4496.0        7066.0    0.0   0.0  
4           2703.0         1378.5        1782.5    0.0   0.0  
..             ...            ...           ...    ...   ...  
241         2900.0         4492.0        5574.0    1.0   1.0  
242         2216.0         2837.0        7186.0    3.0   3.0  
243         1496.0         2939.0        6166.0    1.0   1.0  
244         6026.0         5205.0       11823.0    8.0   8.0  
245         1782.0         2592.0        4931.0    3.0   3.0  

[246 rows x 68 columns]

In [15]:
a_1 = a_[['for_4050', 'for_합계', 'for_u20', 'for_2030', 'single_2030',
       'single_f_o60', 'single_o60', 'single_f_합계', 'popu_2030', '가해_10대이하',
       'theft_violence_y', 'popu_o60', 'single_합계', 'robber_murder_y',
       'for_o60', 'report_sx_여성', '피해_80대이상', 'single_f_4050', 'vio_cnt',
       'robber_murder_x']]

In [17]:
a_2 = a_[['robber_murder_y', 'single_o60', '가해_10대이하', 'for_u20',
       'robber_murder_x', 'for_합계', '피해_80대이상', 'vio_cnt', 'single_합계',
       'single_f_합계', 'theft_violence_y', 'theft_violence_x', 'for_o60',
       'single_4050', 'single_f_4050', 'popu_o60', 'single_f_o60',
       'report_sx_여성', 'for_2030', 'for_4050']]

In [18]:
a_3 = a_[['robber_murder_y', 'single_f_2030', '가해_80대이상', 'single_2030',
       'single_4050', '피해_70대', 'traffic_x', 'adult', 'robber_murder_x',
       'mur_rob_cnt', '가해_70대', 'theft_violence_y', 'for_4050', 'single_합계',
       'for_o60', 'theft_violence_x', '5m_crm_yn', 'for_합계', 'for_2030',
       'bell']]

In [19]:
a_4 = a_[['mur_rob_cnt', 'for_4050', 'for_합계', 'single_4050', 'single_합계', 'cctv',
       'for_2030', 'theft_violence_x', 'for_o60', '가해_여', 'car_cnt']]

In [20]:
a_5 = a_[['single_o60', 'single_f_o60', 'single_f_합계', 'for_2030', 'single_2030',
       'for_합계', 'for_o60', 'popu_o60', 'vio_cnt', 'single_4050', 'popu_2030',
       'for_u20', 'single_f_4050', 'robber_murder_x', 'single_합계', '가해_여',
       'for_4050', '피해_80대이상', 'robber_murder_y', 'theft_violence_x']]

In [22]:
# StandardScaling
scaler1 = StandardScaler()
scaler1.fit(a_1) # scaler에 xdata 학습
a_s1 = scaler1.transform(a_1)
a_s1

array([[-0.44471781, -0.55302633, -0.48898838, ..., -1.70872887,
        -1.08002514,  0.02046058],
       [-0.48941689, -0.51798283, -0.33491247, ..., -1.62995962,
        -1.26140432, -0.69858259],
       [-0.44471781, -0.55302633, -0.48898838, ..., -1.70872887,
        -1.38640889, -1.41762576],
       ...,
       [-0.61571289, -0.7750176 , -0.93463395, ..., -0.79942306,
        -0.76056902,  0.73950375],
       [-0.46327215, -0.35479004, -0.52007993, ...,  0.406598  ,
        -0.07672048, -1.41762576],
       [-0.46179624, -0.57418004, -0.81303144, ..., -0.98410501,
        -0.98198234, -1.41762576]])

In [23]:
# StandardScaling
scaler2 = StandardScaler()
scaler2.fit(a_2) # scaler에 xdata 학습
a_s2 = scaler2.transform(a_2)
a_s2

array([[-0.45018049, -1.54175937, -1.08718846, ..., -1.08891897,
        -0.71525744, -0.44471781],
       [-0.11761472, -1.47523454, -1.17944384, ..., -1.17135971,
        -0.53631115, -0.48941689],
       [-1.44787778, -1.54175937, -1.36395461, ..., -1.58314811,
        -0.71525744, -0.44471781],
       ...,
       [ 0.54751681, -0.0185895 , -0.99493307, ..., -0.7325756 ,
        -0.93101467, -0.61571289],
       [-1.44787778,  1.55945205, -0.16463461, ...,  0.05819103,
         0.02757248, -0.46327215],
       [-1.44787778, -0.48426336, -0.34914538, ..., -1.01686119,
        -0.59565942, -0.46179624]])

In [24]:
# StandardScaling
scaler3 = StandardScaler()
scaler3.fit(a_3) # scaler에 xdata 학습
a_s3 = scaler3.transform(a_3)
a_s3

array([[-0.45018049, -0.97165206, -0.83332757, ..., -0.55302633,
        -0.71525744, -0.59342342],
       [-0.11761472, -0.87776189, -1.19046796, ..., -0.51798283,
        -0.53631115,  0.29943384],
       [-1.44787778, -0.97165206, -0.83332757, ..., -0.55302633,
        -0.71525744, -0.59342342],
       ...,
       [ 0.54751681, -1.0748496 , -0.83332757, ..., -0.7750176 ,
        -0.93101467, -0.14699479],
       [-1.44787778, -0.33515836,  1.30951476, ..., -0.35479004,
         0.02757248,  2.97800563],
       [-1.44787778, -1.02814945, -0.1190468 , ..., -0.57418004,
        -0.59565942,  0.74586247]])

In [25]:
# StandardScaling
scaler4 = StandardScaler()
scaler4.fit(a_4) # scaler에 xdata 학습
a_s4 = scaler4.transform(a_4)
a_s4

array([[-0.68019192, -0.44471781, -0.55302633, ..., -0.38911982,
        -1.11672616, -0.92567779],
       [-0.68019192, -0.48941689, -0.51798283, ..., -0.45704651,
        -1.16852978, -1.4252536 ],
       [-0.68019192, -0.44471781, -0.55302633, ..., -0.38911982,
        -1.20306553, -1.54533513],
       ...,
       [ 1.90901058, -0.61571289, -0.7750176 , ..., -0.59093101,
        -0.85770802, -0.46918467],
       [-0.68019192, -0.46327215, -0.35479004, ..., -0.55745988,
        -0.13245724,  1.05245908],
       [-0.68019192, -0.46179624, -0.57418004, ..., -0.5358021 ,
         0.28197177, -0.66901499]])

In [26]:
# StandardScaling
scaler5 = StandardScaler()
scaler5.fit(a_5) # scaler에 xdata 학습
a_s5 = scaler5.transform(a_5)
a_s5

array([[-1.54175937, -1.55004946, -1.40869518, ..., -1.00033815,
        -0.45018049, -0.40220751],
       [-1.47523454, -1.50210803, -1.32273075, ..., -0.79961777,
        -0.11761472, -0.54579805],
       [-1.54175937, -1.55004946, -1.40869518, ..., -1.4017789 ,
        -1.44787778, -0.88797125],
       ...,
       [-0.0185895 , -0.07250741, -0.85703135, ..., -0.59889739,
         0.54751681, -1.19653815],
       [ 1.55945205,  1.77242259,  0.42477579, ...,  0.80614526,
        -1.44787778, -0.50913663],
       [-0.48426336, -0.47528071, -0.98925329, ...,  1.40830639,
        -1.44787778, -1.57842789]])

In [29]:
# x데이터 설정 2017, 2018 데이터를 학습용, 2019년 데이터를 검증용 데이터셋으로 설정

xtrain1 = a_s2[:-82]
xtest1 = a_s2[-82:]
xtrain2 = a_s2[:-82]
xtest2 = a_s2[-82:]
xtrain3 = a_s3[:-82]
xtest3 = a_s3[-82:]
xtrain4 = a_s4[:-82]
xtest4 = a_s4[-82:]
xtrain5 = a_s5[:-82]
xtest5 = a_s5[-82:]

In [30]:
# y데이터 설정 2017, 2018 데이터를 학습용, 2019년 데이터를 검증용 데이터셋으로 설정

train = a[:-82]
test = a[-82:]

ytrain1 = train['q1']
ytrain2 = train['q2']
ytrain3 = train['q3']
ytrain4 = train['q4']
ytrain5 = train['q5']

ytest1 = test['q1']
ytest2 = test['q2']
ytest3 = test['q3']
ytest4 = test['q4']
ytest5 = test['q5']

In [31]:
a_1.columns

Index(['for_4050', 'for_합계', 'for_u20', 'for_2030', 'single_2030',
       'single_f_o60', 'single_o60', 'single_f_합계', 'popu_2030', '가해_10대이하',
       'theft_violence_y', 'popu_o60', 'single_합계', 'robber_murder_y',
       'for_o60', 'report_sx_여성', '피해_80대이상', 'single_f_4050', 'vio_cnt',
       'robber_murder_x'],
      dtype='object')

In [32]:
# 그리드 서치 패키지
from sklearn.model_selection import GridSearchCV

In [33]:
# 최적 성능을 내는 lasso의 alpha값을 얻기 위해 param_grid 생성

param_grid = {'l1_ratio': np.linspace(0.01, 1.0, 100), 'alpha' : np.linspace(0.001, 1.0, 1000)}

#그리드 서치 설정
#grid_search = GridSearchCV(ElasticNet(), param_grid = param_grid, cv = 10, n_jobs = -1, scoring ='r2')
grid_search = GridSearchCV(ElasticNet(), param_grid = param_grid, cv = 10, n_jobs = -1, scoring ='neg_mean_absolute_error')

#### q1 절도폭력 

In [34]:
# 그리드 서치 후 최고 성능의 모델을 ela1에 저장

grid_search.fit(xtrain1, ytrain1)
ela1 = grid_search.best_estimator_

In [35]:
# MAE 출력

y_pred1 = ela1.predict(xtest1)
mean_absolute_error(ytest1, y_pred1)

2.828463506950819

In [46]:
# 결과

print('alpha =', ela1.alpha)
print('l1_ratio =', ela1.l1_ratio)
print(ela1.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 갯수:', len(a_1.columns[ela1.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_1.columns[ela1.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_1.columns[ela1.coef_.argmin()])

alpha = 0.035
l1_ratio = 1.0
[-0.06184511 -2.33635067 -0.39948134  1.27728687  0.39081619 -0.
 -0.         -1.92386271  0.          1.21651276 -0.38274654  1.99886289
 -1.38972091 -1.65551156  0.          3.32187811  0.         -0.
 -1.60045414 -0.        ]
feature 갯수: 13
가장 강한 양의 상관관계:  report_sx_여성 
가장 강한 음의 상관관계:  for_합계


#### q2 강도살인

In [47]:
# 그리드 서치 후 최고 성능의 모델을 ela2에 저장

grid_search.fit(xtrain2, ytrain2)
ela2 = grid_search.best_estimator_

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 301.5526773095701, tolerance: 0.18037177439024385
  model = cd_fast.enet_coordinate_descent(


In [48]:
# MAE 출력

y_pred2 = ela2.predict(xtest2)
mean_absolute_error(ytest2, y_pred2)

2.874407091957171

In [49]:
# 결과

print('alpha =', ela2.alpha)
print('l1_ratio =', ela2.l1_ratio)
print(ela2.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 갯수:', len(a_2.columns[ela2.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_2.columns[ela2.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_2.columns[ela2.coef_.argmin()])

alpha = 0.001
l1_ratio = 0.35000000000000003
[-0.14874159 -6.52197683 -0.80885577  2.04053491  0.38416593 -1.17548343
  0.0085108  -2.5366865   5.74329762 -4.74341114 -0.72826919  2.136799
 -5.2964221  -4.60422628  2.64362271  5.53829187  3.41264214  0.23370656
 -1.50098838  4.69569028]
feature 갯수: 20
가장 강한 양의 상관관계:  single_합계 
가장 강한 음의 상관관계:  single_o60


In [50]:
# q2 관련 feature
print('상위 30개 feature\n', a_2.columns[sorted(range(len(ela2.coef_)), key=lambda i: abs(ela2.coef_[i]))[-30:]])

상위 30개 feature
 Index(['피해_80대이상', 'robber_murder_y', 'report_sx_여성', 'robber_murder_x',
       'theft_violence_y', '가해_10대이하', 'for_합계', 'for_2030', 'for_u20',
       'theft_violence_x', 'vio_cnt', 'single_f_4050', 'single_f_o60',
       'single_4050', 'for_4050', 'single_f_합계', 'for_o60', 'popu_o60',
       'single_합계', 'single_o60'],
      dtype='object')


#### q3 교통안전

In [51]:
# 그리드 서치 후 최고 성능의 모델을 lasso3에 저장

grid_search.fit(xtrain3, ytrain3)
ela3 = grid_search.best_estimator_

In [52]:
# MAE 출력

y_pred3 = ela3.predict(xtest3)
mean_absolute_error(ytest3, y_pred3)

2.3844019973772177

In [53]:
# 결과

print('alpha =', ela3.alpha)
print('l1_ratio =', ela3.l1_ratio)
print(ela3.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 갯수:', len(a_3.columns[ela3.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_3.columns[ela3.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_3.columns[ela3.coef_.argmin()])

alpha = 0.139
l1_ratio = 1.0
[-0.         -0.          0.         -0.         -0.59846375  0.45957675
 -0.35440786 -0.          0.          0.          0.41455889 -0.
 -0.01561904 -0.         -0.          0.          0.         -0.
 -0.09377313 -0.        ]
feature 갯수: 6
가장 강한 양의 상관관계:  피해_70대 
가장 강한 음의 상관관계:  single_4050


In [54]:
# q3 관련 feature
print('상위 30개 feature\n', a_3.columns[sorted(range(len(ela3.coef_)), key=lambda i: abs(ela3.coef_[i]))[-30:]])

상위 30개 feature
 Index(['robber_murder_y', 'single_f_2030', '가해_80대이상', 'single_2030', 'adult',
       'robber_murder_x', 'mur_rob_cnt', 'theft_violence_y', 'single_합계',
       'for_o60', 'theft_violence_x', '5m_crm_yn', 'for_합계', 'bell',
       'for_4050', 'for_2030', 'traffic_x', '가해_70대', '피해_70대', 'single_4050'],
      dtype='object')


#### q4 법질서 준수도

In [55]:
# 그리드 서치 후 최고 성능의 모델을 lasso4에 저장

grid_search.fit(xtrain4, ytrain4)
ela4 = grid_search.best_estimator_

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 516.1503833083053, tolerance: 0.27214904878048785
  model = cd_fast.enet_coordinate_descent(


In [56]:
# MAE 출력

y_pred4 = ela4.predict(xtest4)
mean_absolute_error(ytest4, y_pred4)

2.743810830654499

In [57]:
# 결과

print('alpha =', ela4.alpha)
print('l1_ratio =', ela4.l1_ratio)
print(ela4.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 갯수:', len(a_4.columns[ela4.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_4.columns[ela4.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_4.columns[ela4.coef_.argmin()])

alpha = 0.001
l1_ratio = 1.0
[ 1.40036205 -5.36434607  9.38044717 -2.19525035  1.52197704  0.84764676
 -3.6164628  -3.85464281 -1.13992311  1.32328656  3.26299134]
feature 갯수: 11
가장 강한 양의 상관관계:  for_합계 
가장 강한 음의 상관관계:  for_4050


In [58]:
# q3 관련 feature
print('상위 30개 feature\n', a_4.columns[sorted(range(len(ela4.coef_)), key=lambda i: abs(ela4.coef_[i]))[-30:]])

상위 30개 feature
 Index(['cctv', 'for_o60', '가해_여', 'mur_rob_cnt', 'single_합계', 'single_4050',
       'car_cnt', 'for_2030', 'theft_violence_x', 'for_4050', 'for_합계'],
      dtype='object')


#### q5 전반적 안전도

In [59]:
# 그리드 서치 후 최고 성능의 모델을 lasso4에 저장

grid_search.fit(xtrain5, ytrain5)
ela5 = grid_search.best_estimator_

In [60]:
# MAE 출력

y_pred5 = ela5.predict(xtest5)
mean_absolute_error(ytest5, y_pred5)

2.345048879490055

In [61]:
# 결과

print('alpha =', ela5.alpha)
print('l1_ratio =', ela5.l1_ratio)
print(ela5.coef_) # Ridge 회귀분석으로 나온 weghit값
print('feature 갯수:', len(a_5.columns[ela5.coef_ != 0]))
print('가장 강한 양의 상관관계: ',a_5.columns[ela5.coef_.argmax()], '\n가장 강한 음의 상관관계: ', a_5.columns[ela5.coef_.argmin()])

alpha = 0.009000000000000001
l1_ratio = 0.01
[-1.25867186  0.29647243  1.05212064 -1.1459099   0.17623391 -0.45722915
 -0.40751588  2.12251136 -1.64077225 -2.22012702  0.28920447  1.41776717
  0.57067653  0.5053599  -0.54931721  1.1830857  -0.2213457  -0.50944601
 -0.20606122  0.46416553]
feature 갯수: 20
가장 강한 양의 상관관계:  popu_o60 
가장 강한 음의 상관관계:  single_4050


In [62]:
print('feature: ', a_5.columns[ela5.coef_ != 0])

feature:  Index(['single_o60', 'single_f_o60', 'single_f_합계', 'for_2030', 'single_2030',
       'for_합계', 'for_o60', 'popu_o60', 'vio_cnt', 'single_4050', 'popu_2030',
       'for_u20', 'single_f_4050', 'robber_murder_x', 'single_합계', '가해_여',
       'for_4050', '피해_80대이상', 'robber_murder_y', 'theft_violence_x'],
      dtype='object')


In [63]:
# q5 관련 feature
print('상위 30개 feature\n', a_5.columns[sorted(range(len(ela5.coef_)), key=lambda i: abs(ela5.coef_[i]))[-30:]])

상위 30개 feature
 Index(['single_2030', 'robber_murder_y', 'for_4050', 'popu_2030',
       'single_f_o60', 'for_o60', 'for_합계', 'theft_violence_x',
       'robber_murder_x', '피해_80대이상', 'single_합계', 'single_f_4050',
       'single_f_합계', 'for_2030', '가해_여', 'single_o60', 'for_u20', 'vio_cnt',
       'popu_o60', 'single_4050'],
      dtype='object')
